# Regressão Logística
Usei o Kernel do Kaggle

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


### O dataset Mines vs Rocks já está no Kaggle, mas pode ser baixado daqui:  
https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+%28Sonar%2C+Mines+vs.+Rocks%29


In [ ]:
input_data = pd.read_csv('/kaggle/input/mines-vs-rocks/sonar.all-data.csv', header=None)
input_data.head() # uma olhada no dataset

Conta quantas minas e quantas rochas tem no dataset  #minas 0    #rochas 1

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  #codifica as letras M e R em 0 e 1
y = input_data[60].copy()
y = LabelEncoder().fit_transform(y)

X_df = input_data.copy()
X_df.drop([60], inplace=True, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_df.values, y, test_size=0.2, shuffle=True, random_state=42)

print('Training data and target sizes: \n{}, {}'.format(X_train.shape,y_train.shape))
print('Test data and target sizes: \n{}, {}'.format(X_test.shape,y_test.shape))

# Criação do Modelo

Numa aplicação real, não pode haver falso negativo, pois seria desastroso identificar uma mina como rocha. Falsos positivos são aceitáveis. Assim, coloquei peso maior (2.975) para a classe 0de forma que não houvesse falsos negativos. Valor foi obtido com tentativa e erro

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


class_logistic = LogisticRegression(solver='liblinear', max_iter=100,class_weight={0:2.975}) # liblinear deu o melhor resultado
class_logistic.fit(X_train, y_train)

y_pred = class_logistic.predict(X_test)
print("Accuracy of model = %2f%%" % (accuracy_score(y_test, y_pred )*100))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, y_pred))

Accuracy of model = 80.952381%
Confusion matrix:
[[26  0]
 [ 8  8]]

## Visualização dos coeficientes
São esses coeficientes que serão transferidos para o código C do Arduino.

In [ ]:
print(class_logistic.intercept_,class_logistic.coef_)

### Implementando o algoritmo de regressão logística
Somente para verificação. Comparamos os valores obtidos com o classificador  class_logistic com o algoritmo implementado. Encontraremos valores semelhantes 

In [ ]:
intercept=class_logistic.intercept_
coefs=np.array(class_logistic.coef_)

p=np.exp(-(np.dot(X_test[17].transpose(), coefs)+intercept))  #predição da amostra número 17
p

In [ ]:
class_logistic.predict_proba(X_test[17])  #predição da amostra 17 usando o classificador criado no scikit-learn

### Download dos coeficientes
Algoritmo verificado, basta implementarmos em C no Arduíno, utilizando os coeficientes. Para facilitar, usaremos o código abaixo e exportamos para um arquivo texto. No Kaggle, é só fazer o commit e buscar os dados em Output.

In [ ]:
np.savetxt('RegressaoLogistica.txt',class_logistic.coef_, delimiter=',', fmt='%f') #salva os coeficientes